In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
data = pd.read_csv('data/lesser_model_data.csv')

In [10]:
"""
IMPORTANT NOTE Y'ALL::maybe once we have all three models we can move the get_lesser_model_states
to some sort of get states code and just make this a place to get general probabilities 
(and call get_states and probabilities functions from a network specific circuit builder directly,
OR a general circuit builder would be the dream!!)
"""

"\nIMPORTANT NOTE Y'ALL::maybe once we have all three model we can move the get_lesser_model_states\nto some sort of get states code and just make this a place to get general probabilities \n(and call get_states and probabilities functions from a network specific circuit builder directly,\nOR a general circuit builder would be the dream!!)\n"

In [11]:
def get_lesser_model_states():
    statedataStayHome = {'MarHome' : data['MarHome'], 'AprHome' : data['AprHome'], 'MayHome' : data['MayHome'], 'JunHome' : data['JunHome']}
    statesStayHome = pd.DataFrame(data=statedataStayHome)

    statedataTests = {'MarTest' : data['MarTest'], 'AprTest' : data['AprTest'], 'MayTest' : data['MayTest'], 'JunTest' : data['JunTest']}
    statesTests = pd.DataFrame(data=statedataTests)

    statedataCases = {'MarCases' : data['MarCases'], 'AprCases' : data['AprCases'], 'MayCases' : data['MayCases'], 'JunCases' : data['JunCases']}
    statesCases = pd.DataFrame(data=statedataCases) # 0 = increasing. 1 = flat or decreasing
    
    return statesStayHome, statesTests, statesCases

In [62]:
def get_probabilities(state_data):
    ############################################
    # USE THIS FUNCTION TO FIND THE PROBABILITIES FOR AN INDIVIDUAL NODE IN THE NETWORK
    
    ### INPUT ###
    # state_data: array or pandas dataframe of any dimension and shape containing integers
    #         indicating the state corresponding to individual data points (note: for an
    #         n-state system the array can contain the integers 0, 1, ..., n-1, but must
    #         contain n-1 at least once)

    ### OUTPUT ###
    # probs: array of length n with probabilities for states 0 through n-1 in increasing order
    ############################################
    state_data = np.ndarray.flatten(np.array(state_data)) #now the input can be an array or pandas dataframe of any dimension
    num_total = len(state_data)  #total number of data points with which to calculate probabilities
    s = max(state_data) + 1  #number of different states will be the maximum integer included in the state data plus one

    probs = np.zeros(s)
    for i in range(s): #loop through different allowed states
        probs[i] = np.shape(np.where(state_data == i))[1]/num_total

    assert round(np.sum(probs), 3) == 1. 
    
    return probs

In [5]:
### IN PROGRESS OF GENERALIZING THIS TO m PARENT nodes with s_i states (parent 0 has s_0 states, parent 1 has s_1 states, etc.)
### THE OLD VERSION IS IN ARCHIVED FUNCTIONS NOTEBOOK!

def get_conditional_probability(*state_datas):
    ############################################
    ### THIS FUNCTION CALCULATES CONDITIONAL PROBABILITIES FOR CHILD NODE
    ### THAT HAS s_m STATES AND m PARENT NODES EACH WITH s_i STATES WHERE i = 0, ..., m-1
    
    ### INPUT ###
    # this function takes m+1 inputs
    # each input should be an array or pandas dataframe of the same dimension and shape containing integers
    #         indicating the state corresponding to individual data points for a node
    # the first m inputs should be the state data of the parent nodes of one particular child node
    # the last input should be the state data for that child node
    
    ### OUTPUT ###
    # a dictionary of conditional probabilities...hmmm need to think this through???
    
    m = len(state_datas) - 1
    
    s = np.zeros(m+1)
    for i in range(m+1)
        state_datas[i] = np.ndarray.flatten(np.array(state_datas[i]))
        s[i] = max(state_datas[i]) + 1
    
    #figure out how to generate a list of strings: ["0|00...00", 0|00...01, ... "s_m|s_(m-1)s_(m-2)...s1s0"]
    
    """
    for s in s_m:
        #Add up cases where child = s
        ind = np.where(Cstates == 1)[0] #index flattened array with child in state s

        #gonna need to figure this out...how to have m-1 nested ifs, change notation, store in an array of nums
        for i in range(len(indC_one_col)): # loop through all times C=1
            A = Astates.iloc[indC_one_row[i], indC_one_col[i]]
            B = Bstates.iloc[indC_one_row[i], indC_one_col[i]]
            if A == 1:
                if B == 1:
                    num111 += 1  #C=1, B=1, A=1
                    num11 += 1  #B=1, A=1
                elif B == 0:
                    num101 += 1  #C=1, B=0, A=1
                    num01 += 1  #B=0, A=1
            elif A == 0:
                if B == 1:
                    num110 += 1  #C=1, B=1, A=0
                    num10 += 1  #B=1, A=0
                elif B == 0:
                    num100 += 1  #C=1, B=0, A=0
                    num00 += 1  #B=0, A=0
        
   
    # Calculate conditional probabilities
    num_total = np.shape(Astates)[0] * np.shape(Astates)[1]
    
    # the variable names are in the order CBA (cases, testing, stay at home)
    P111 = num111/num11
    P101 = num101/num01
    P110 = num110/num10
    P100 = num100/num00
    P011 = num011/num11
    P001 = num001/num01
    P010 = num010/num10
    P000 = num000/num00
    
    states = np.array(['0|00','1|00','0|01','1|01','0|10','1|10','0|11','1|11']) #I believe this is the order Matt was talking about with top to bottom, left to right in the table from the paper
    conditional_probabilities = np.array([P000,P100,P001,P101,P010,P110,P011,P111]) #in this order, the first two, second two, etc. should each add to 1 as pairs (columns in the paper's table)
    
    # Make graph for circuit input
    ProbA1, ProbA0 = get_probabilities(Astates)
    ProbB1, ProbB0 = get_probabilities(Bstates)
    
    graph = { 'StayAtHome': ([], [ProbA0, ProbA1]), # P(A = 0), P(A = 1)
                'Testing': ([], [ProbB0, ProbB1]),  #P(B = 0), P(B = 1)
                #P(C=0|A=0,B=0), P(C=1|A=0,B=0), P(C=0|A=0,B=1), P(C=1|A=0,B=1), P(C=0|A=1,B=0), P(C=1|A=1,B=0), P(C=0|A=1,B=1), P(C=1|A=1,B=1)
                'Cases': (['StayAtHome','Testing'], [P000, P100, P010, P110, P001, P101, P011, P111])
              }
    
    return states, conditional_probabilities, graph
    """

In [6]:
# example: results from running simple model on simulator: 
# {'000': 2783, '001': 1240, '100': 603, '111': 815, '110': 294, '010': 1712, '101': 485, '011': 260}

def get_marginal_0probabilities(state_counts):
    #state_counts: dict, counts for each state from network result (should have 2^n entries)
    #marg_probs: array of length n, marginal probabilities that each qubit is 0,
        #from most significant to least significant qubit
    
    n = len(list(state_counts.keys())[0]) #number of qubits
    prob = np.zeros(n)
    total = sum(state_counts.values())

    for i in range(n):
        for key in state_counts:
            if int(key[i]) == 0:
                prob[i] += state_counts[key]
        prob[i] = prob[i]/total
    
    return prob




These calculations agree with marginal_probabilities function, yay !

PC0_calc = (2783+1240+1712+260)/(2783+1240+603+815+294+1712+485+260)

print(PC0_calc)

PB0_calc = (2783+1240+485+603)/(2783+1240+603+815+294+1712+485+260)

print(PB0_calc)

PA0_calc = (2783+603+294+1712)/(2783+1240+603+815+294+1712+485+260)

print(PA0_calc)


In [7]:
# get marginal probabilities for simulation results - lesser model

results = {'000': 2783, '001': 1240, '100': 603, '111': 815, '110': 294, '010': 1712, '101': 485, '011': 260}
margProb = marginal_probabilities(results)
margCompare = marginal_probabilities_general(results)
print(margProb)
print(margCompare)

(0.7318115234375, 0.6239013671875, 0.658203125)
[0.73181152 0.62390137 0.65820312]


In [6]:
def func(**counts):
    return counts["c001"]

In [7]:
func(c000=4928, c001=82743, c100=48937, c111=9288842)

82743